In [1]:
import numpy as np
from scipy.stats import unitary_group
#fixed!!
nDim = 2
Ns = 2
Nc = 2

Nx = 4
Nt = 4
lattice_volume = Nt * Nx

In [2]:
Pauli = []
Pauli.append(np.array([[0, 1], [1, 0]]))
Pauli.append(np.array([[0,-1j], [1j, 0]]))
Pauli.append(np.array([[1, 0], [0, -1]]))
Id = np.array([[1, 0], [0, 1]])

In [3]:
def Transpose(array):
    axes = np.arange(len(array.shape))
    axes[-2:] = np.flip(axes[-2:]) 
    return np.transpose(array, axes = axes)

In [31]:
def ConjugateTranspose2(array):
    axes = np.arange(len(array.shape))
    axes[-2:] = np.flip(axes[-2:]) 
    return np.conjugate(np.transpose(array, axes = axes))

In [5]:
def RandomSU2matrix():
    tmp = np.random.rand(3)
    a = np.sqrt(sum(tmp*tmp))
    tmp /= a
    out = np.array([[0.j,0.j],[0.j,0.j]])
    for i in range(3):
        out += Pauli[i] * tmp[i]
    out *= np.sin(a) * 1j
    return Id * np.cos(a) + out

Dirac Lagrangian on the lattice using Wilson fermions:

$\overline{\psi}_x D_{xy}[U] \psi_y = \overline{\psi}_x \left[ (m_q+2) \delta_{xy} -\frac{1}{2}\sum_{\mu} \left(\Gamma_{+\mu} U_\mu(x) \delta_{x+\hat\mu,y} + \Gamma_{-\mu} U^\dagger_\mu(x-\hat\mu) \delta_{x-\hat\mu,y} \right) \right] \psi_y$,

where $\Gamma_{\pm \mu} = \mathbb{1} \mp \gamma_\mu$, $\gamma_\mu$ are the Dirac $\gamma$-matrices, $U_\mu(x)$ is the gauge link at site $x$ pointing at direction $\mu$, and $\hat\mu$ is a unit vector pointing in direction $\mu$.

Note that the $\Gamma$'s are $2 \times 2$ matrices in 2 spacetime dimensions. Dirac and colour indices have been suppressed in the above formula for clarity.

Calculating D, and inverting D is numerically slow, therefore we use the approximation where we can expand the inverse acting on a vector by a sum over applying D multiple times to this same vector. For this we need to Calculate D, which is sparse, and then compute the Matrix-Vector multiplication, which can be done for CSR Sparse matrices. Other way to do this (maybe not possible in the optical hardware?) is to directly apply D to the vector, and this is also implemented

In [6]:
psi = np.random.randn(lattice_volume, 2, 2)

In [7]:
gauge_links = np.zeros((lattice_volume, nDim, 2,2),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = RandomSU2matrix()

In [8]:
def LinearToLattice(array, Nt, Nx):
    return np.reshape(array, np.roll(np.append(np.array(array[0].shape), [Nx,Nt]),2))
def LatticeToLinear(array, lattice_volume):
    return np.reshape(array, np.roll(np.append(np.array(array[0,0].shape), lattice_volume),1))

# Sparse Matrix Multiplication

In [9]:
def CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli):
    #Delta_x_y, dependent only on the onsite
    diagonal = (m+2)
    
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]
    
    gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
    gauge_links = LatticeToLinear(gauge_links, lattice_volume) * (-0.5)
    gauge_links_shifted_t = LatticeToLinear(gauge_links_shifted_t, lattice_volume) * (-0.5)
    gauge_links_shifted_x = LatticeToLinear(gauge_links_shifted_x, lattice_volume) * (-0.5)
    
    
    #In each row they are 2^d (next Neighbors) + 1 (onsite)
    row_index = np.arange(0,(2**nDim+1)*lattice_volume+1, 2**nDim+1)


    col_index = np.array([])
    values_spinors = np.array([])
    values_colors = np.array([])
    for i in range(lattice_volume):
        #Just moving in time and space to the next neighbors considering periodic boundary
        col_index = np.append(col_index,
                                 np.array([(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt
                                           ,(np.floor(i/4)*4)+((i%Nx)-1)%Nx, i,
                                           (np.floor(i/4)*4)+((i%Nx)+1)%Nx, (i%Nx) +((np.floor((i/Nt))+1)%Nt)*Nt]))
        #Choose the right values depending on direction in space-time, according to the order from above in col_index
        
        values_spinors = np.append(values_spinors, np.array([offdiagonal_spinor_t_minus, 
                                         offdiagonal_spinor_x_minus, np.sqrt(diagonal)*np.identity(Ns), 
                                                             offdiagonal_spinor_x_plus, offdiagonal_spinor_t_plus]))
        #We have to use sqrt of diagonal instead of diagonal, because we perform two matrix multiplication
        
#         values_colors = np.append(values_colors, np.array([gauge_links[(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt,0].T, 
#                                          gauge_links[(np.floor(i/4)*4)+((i%Nx)-1)%Nx,1].T, diagonal*np.identity(Nc), 
#                                                              gauge_links[i,1], 
#                                                            offdiagonal_spinor_t_plus]))
        values_colors = np.append(values_colors, np.array([ConjugateTranspose(gauge_links_shifted_t[i]), 
                                         ConjugateTranspose(gauge_links_shifted_x[i]), np.sqrt(diagonal)*np.identity(Nc), 
                                                             gauge_links[i,1], gauge_links[i,0]]))
    
        #I feel the transpose should be opposite, or idk
        
        #Maybe it's smart to add a value_diagonal, and separate all three, we could eliminate the diagonal term in col
        #and then we can choose the correct gauge link using the col_index. Or maybe we could move the gauge_link to right
        #The problem with the first approach, is that the positive u dont shift the gauge link. Therefore, maybe only
        #considere specific elements of col_index, namely the minus part and the diagonal part. 
        
        
    values_spinors = np.reshape(values_spinors, (lattice_volume*(2**nDim+1), Ns,Ns))
    values_colors = np.reshape(values_colors, (lattice_volume*(2**nDim+1), Nc,Nc))
    return row_index.astype(int), col_index.astype(int), values_spinors, values_colors

In [10]:
def CalculateD_flatten(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli):
    #Delta_x_y, dependent only on the onsite
    diagonal = (m+2)
   
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]
   
    gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
    gauge_links = LatticeToLinear(gauge_links, lattice_volume) * (-0.5)
    gauge_links_shifted_t_T = ConjugateTranspose(LatticeToLinear(gauge_links_shifted_t, lattice_volume) * (-0.5)) #.T correct?
    gauge_links_shifted_x_T = ConjugateTranspose(LatticeToLinear(gauge_links_shifted_x, lattice_volume) * (-0.5))
   
    offdiag_x_plus = np.kron(gauge_links[:,1,:,:],offdiagonal_spinor_x_plus) #Careful with U dimension t,x
    offdiag_x_minus = np.kron(gauge_links_shifted_x_T,offdiagonal_spinor_x_minus)
    offdiag_t_plus = np.kron(gauge_links[:,0,:,:],offdiagonal_spinor_t_plus)
    offdiag_t_minus = np.kron(gauge_links_shifted_t_T,offdiagonal_spinor_t_minus)

    #In each row they are 2^d (next Neighbors) + 1 (onsite)
    row_index = np.arange(0,(2**nDim+1)*lattice_volume+1, 2**nDim+1)


    col_index = np.array([])
    values = np.array([])

    for i in range(lattice_volume):
        #Just moving in time and space to the next neighbors considering periodic boundary
        col_index = np.append(col_index,
                                 np.array([(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt
                                           ,(np.floor(i/4)*4)+((i%Nx)-1)%Nx, i,
                                           (np.floor(i/4)*4)+((i%Nx)+1)%Nx, (i%Nx) +((np.floor((i/Nt))+1)%Nt)*Nt]))
        #Choose the right values depending on direction in space-time, according to the order from above in col_index
       
        values = np.append(values, np.array([offdiag_t_minus[i],
                                         offdiag_x_minus[i], diagonal*np.identity(Ns*Nc),
                                                             offdiag_x_plus[i], offdiag_t_plus[i]]))

       
    values = np.reshape(values, (lattice_volume*(2**nDim+1), Ns*Nc,Ns*Nc))
    return row_index.astype(int), col_index.astype(int), values


In [11]:
def SparseMatrixVectorMultiplication(rows, cols, vals, input_vec):
    out_vec = np.zeros(input_vec.shape,dtype = 'complex_')
    for i in range(input_vec[:,0].size):
        row_elem = rows[i]
        num_elements = rows[i+1] - rows[i]

        output = np.zeros(input_vec[0].shape,dtype = 'complex_')
        for j in range(num_elements):
            output = output + np.matmul(vals[row_elem + j],  input_vec[cols[row_elem + j]])
        out_vec[i] = output
    
    return out_vec

In [12]:
def SparseMatrixVectorMultiplication2(rows, cols, values_spinors, values_colors, input_vec):
    out_vec = np.zeros(input_vec.shape,dtype = 'complex_')
    for i in range(input_vec[:,0,0].size):
        row_elem = rows[i]
        num_elements = rows[i+1] - rows[i]

        output = np.zeros(input_vec[0].shape,dtype = 'complex_')
        for j in range(num_elements):
            output = output + np.matmul(np.matmul(values_colors[row_elem + j],  input_vec[cols[row_elem + j]]),
                                        values_spinors[row_elem + j].T)
        out_vec[i] = output
    
    return out_vec

#### Test random U

In [13]:
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)
rows, cols, values = CalculateD_flatten(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new_flatten = SparseMatrixVectorMultiplication(rows, cols, values, psi_flatten)
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication2(rows, cols, values_spinors, values_colors, psi)

In [14]:
psi_new_reshaped = np.reshape(psi_new_flatten, (lattice_volume,Nc,Ns))
np.sqrt(np.sum((psi_new-psi_new_reshaped)**2))

(5.134106308878057e-15+0j)

#### Test unitary U

In [15]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = unitary_group.rvs(Nc)
        
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
rows, cols, values = CalculateD_flatten(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new_flatten = SparseMatrixVectorMultiplication(rows, cols, values, psi_flatten)
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication2(rows, cols, values_spinors, values_colors, psi)
psi_new_reshaped = np.reshape(psi_new_flatten, (lattice_volume,Nc,Ns))
np.sqrt(np.sum((psi_new-psi_new_reshaped)**2))

(4.91105344001887e-15+5.019677262575644e-18j)

# Apply D directly

In [32]:
def applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    if len(gauge_links) ==lattice_volume:
        gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    if len(psi) ==lattice_volume:
        psi = LinearToLattice(psi, Nt,Nx)

    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]

    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)

    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    psi_shifted_m_t = np.roll(psi, 1, axis = 0)
    psi_shifted_p_t = np.roll(psi, -1, axis = 0)
    psi_shifted_m_x = np.roll(psi, 1, axis = 1)
    psi_shifted_p_x = np.roll(psi, -1, axis = 1)


    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    time_contribution_m = np.matmul(ConjugateTranspose2(gauge_links_shifted_t), psi_shifted_m_t)
    time_contribution_p = np.matmul(gauge_links[:,:,0,:,:], psi_shifted_p_t)
    space_contribution_m = np.matmul(ConjugateTranspose2(gauge_links_shifted_x), psi_shifted_m_x)
    space_contribution_p = np.matmul(gauge_links[:,:,1,:,:], psi_shifted_p_x)

    # Estos matmuls son diferentes a los matmul
    time_contribution_m = np.matmul(time_contribution_m, Transpose(offdiagonal_spinor_t_minus))
    time_contribution_p = np.matmul(time_contribution_p, Transpose(offdiagonal_spinor_t_plus))
    space_contribution_m = np.matmul(space_contribution_m, Transpose(offdiagonal_spinor_x_minus))
    space_contribution_p = np.matmul(space_contribution_p, Transpose(offdiagonal_spinor_x_plus))
    

    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* (time_contribution_m + 
                                       time_contribution_p + space_contribution_m + space_contribution_p)
    
    return psi_new

In [33]:
def applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    if len(gauge_links) ==lattice_volume:
        gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    if len(psi) ==lattice_volume:
        psi = LinearToLattice(psi, Nt,Nx)

    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]

    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
   
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]


   

   
   
   
    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    psi_shifted_m_t = np.roll(psi, 1, axis = 0)
    psi_shifted_p_t = np.roll(psi, -1, axis = 0)
    psi_shifted_m_x = np.roll(psi, 1, axis = 1)
    psi_shifted_p_x = np.roll(psi, -1, axis = 1)

   
    offdiag_x_plus = np.kron(gauge_links[:,:,1,:,:],offdiagonal_spinor_x_plus) #Careful with U dimension t,x
    offdiag_x_minus = np.kron(ConjugateTranspose2(gauge_links_shifted_x),offdiagonal_spinor_x_minus)
    offdiag_t_plus = np.kron(gauge_links[:,:,0,:,:],offdiagonal_spinor_t_plus)
    offdiag_t_minus = np.kron(ConjugateTranspose2(gauge_links_shifted_t),offdiagonal_spinor_t_minus)
   

    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    time_contribution_m = np.matmul(offdiag_t_minus, psi_shifted_m_t[:,:,:,None]) #Do this without None
    time_contribution_p = np.matmul(offdiag_t_plus, psi_shifted_p_t[:,:,:,None])
    space_contribution_m = np.matmul(offdiag_x_minus, psi_shifted_m_x[:,:,:,None])
    space_contribution_p = np.matmul(offdiag_x_plus, psi_shifted_p_x[:,:,:,None])



    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* np.squeeze((time_contribution_m +  #Squeeze is weird!
                                       time_contribution_p + space_contribution_m + space_contribution_p))
   
    return psi_new

#### Test random U

In [18]:
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
psi_new2_flatten = applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi_flatten)

psi_new2_reshaped = np.reshape(psi_new2_flatten, (Nt,Nx,Nc,Ns))
np.sqrt(np.sum((psi_new2 - psi_new2_reshaped)**2))

4.234886024692275e-15

#### Test unitary U

In [19]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = unitary_group.rvs(Nc)
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
psi_new2_flatten = applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi_flatten)

psi_new2_reshaped = np.reshape(psi_new2_flatten, (Nt,Nx,Nc,Ns))
np.sqrt(np.sum((psi_new2 - psi_new2_reshaped)**2))

(1.023645876696267e-15+1.2041226291908005e-17j)

In [20]:
def MatMul(matrix1, matrix2, shape1, shape2):
    first_index = 0
    while len(shape1)>len(shape2):
        shape2.append(1)
    if shape1[0] == shape2[0]:
        dimension_flattened1 = shape1[1]*shape1[2]
        dimension_flattened2 = shape2[1]*shape2[2]
        dimension_flattened_result = shape1[1]*shape2[2]

        matrix_result = np.zeros((shape1[0]* dimension_flattened_result), dtype = 'complex_')

        for lattice_number in range(shape1[0]):
            for first_index in range(shape1[1]):
                for second_index in range(shape2[2]):
                    start_index1 = lattice_number*dimension_flattened1
                    array1 = matrix1[start_index1 + first_index*shape1[2] : start_index1 + (first_index+1)*shape1[2]]


                    #Here, the second_index should be used for a general matrix multiplication, if needed
                    start_index2 = lattice_number*dimension_flattened2
                    array2 = matrix2[start_index2 + second_index*shape2[1] : start_index2 + (second_index+1)*shape2[1]]


                    start_index = (lattice_number*dimension_flattened_result) +first_index*shape2[2]
                    end_index = (lattice_number*dimension_flattened_result) + (first_index+1)*shape2[2]
                    matrix_result[start_index : end_index] = np.dot(array1,array2)


            
    return matrix_result


dim1 = 56
dim2 = 100
matrix1 = np.random.randn(16*dim1*dim2)
matrix2 = np.random.randn(16*dim2)
shape1 = [16,dim1,dim2]
shape2 = [16,dim2]

matrix1_rs = np.reshape(matrix1,shape1)
matrix2_rs = np.reshape(matrix2,shape2)

matrix_result = MatMul(matrix1, matrix2, shape1, shape2)
matrix_result_reshaped = np.reshape(matrix_result, (16,dim1,1))
matrix_result_matmul = np.matmul(matrix1_rs,matrix2_rs[:,:,None])
print(np.sqrt(np.sum((matrix_result_matmul - matrix_result_reshaped)**2)))
print(matrix_result.shape)

(7.635470388962609e-14+0j)
(896,)


In [21]:
def Roll_t(matrix,shape,roll, Nt,Nx):
    if len(shape)==3:
        shape.append(1)
    matrix_result = np.zeros(matrix.shape, dtype = 'complex_')
    rolled_indices = np.roll(np.arange(Nt),roll)
    size0 = shape[1]*shape[2]*shape[3]
    for t in range(Nt):
        start_index = t*size0
        end_index = (t+1)*size0

        start_index_rolled = rolled_indices[t] *size0
        end_index_rolled = (rolled_indices[t]+1) *size0
        matrix_result[start_index : end_index] = matrix[start_index_rolled: end_index_rolled]
    return matrix_result

Nt2 = 2
Nx2 = 10
lattice_volume2 = Nt2*Nx2
ind1 = 5
ind2 = 1

matrix = np.random.randn(lattice_volume2*ind1*ind2)
shape = [Nt2,Nx2,ind1]
matrixrs = np.reshape(matrix, shape)

matrix_result = Roll_t(matrix,shape,-1,Nt2,Nx2)
matrix_result_reshaped = np.reshape(matrix_result, shape)
matrix_rolled = np.reshape(np.roll(matrixrs,-1,axis = 0),shape)
np.sqrt(np.sum((matrix_result_reshaped - matrix_rolled)**2))

0j

In [22]:
def Roll_x(matrix, shape, roll, Nt,Nx):
    if len(shape)==3:
        shape.append(1)
    matrix_result = np.zeros(matrix.shape, dtype = 'complex_')
    
    size_matrix = shape[2]*shape[3]
    size0 = shape[1]*size_matrix
    
    indices = np.arange(Nx)
    rolled_indices = np.roll(indices,roll)
    
    for t in range(Nt):
        for x in range(Nx):
            start_index = t*size0
            end_index = (t+1)*size0
            array_t = matrix[start_index: end_index]
            
            start_index_x = x*size_matrix
            end_index_x = (x+1)*size_matrix
            start_index_rolled = rolled_indices[x] *size_matrix
            end_index_rolled = (rolled_indices[x]+1) *size_matrix
            rolled_array = array_t[start_index_rolled: end_index_rolled]

            matrix_result[start_index+start_index_x : start_index+ end_index_x] = rolled_array
    return matrix_result

Nt2 = 4
Nx2 = 4
ind1 = 6
ind2 = 6

matrix = np.random.randn(Nt2*Nx2*ind1*ind2)
#matrix = np.arange(Nt2*Nx2*ind1*ind2)
shape = [Nt2,Nx2,ind1,ind2]

matrixrs = np.reshape(matrix, shape)

matrix_rolled = np.roll(matrixrs,1,axis = 1)
matrix_result = Roll_x(matrix,shape,1,Nt2,Nx2)
matrix_result_reshaped = np.reshape(matrix_result, shape)
np.sqrt(np.sum((matrix_result_reshaped - matrix_rolled)**2))

0j

In [23]:
def Roll_x2(matrix,shape,roll, Nt,Nx):
    if len(shape)==3:
        shape.append(1)
    matrix_result = np.zeros(matrix.shape, dtype = 'complex_')
    
    size_matrix = shape[2]*shape[3]
    size0 = shape[1]*size_matrix
    
    indices_t = np.arange(Nt) *size0
    
    indices_x = np.arange(Nx)*size_matrix
    rolled_indices_x = np.roll(indices_x,roll)
    
    indices = np.arange(size_matrix)
     
    rolled_combined_indices = ((indices_t[:,None] + rolled_indices_x[None,:])[:,:,None] + indices[None,:]).flatten()
    #This idea works for a scalar value for each 


    return matrix[rolled_combined_indices]

Nt2 = 16
Nx2 = 20
ind1 = 18
ind2 = 3

matrix = np.random.randn(Nt2*Nx2*ind1*ind2)
#matrix = np.arange(Nt2*Nx2*ind1*ind2)
shape = [Nt2,Nx2,ind1,ind2]

matrixrs = np.reshape(matrix, shape)

matrix_rolled = np.roll(matrixrs,-1,axis = 1)
matrix_result = Roll_x2(matrix,shape,-1,Nt2,Nx2)
matrix_result_reshaped = np.reshape(matrix_result, shape)
np.sqrt(np.sum((matrix_result_reshaped - matrix_rolled)**2))

0.0

In [24]:
def Kron(matrix1,matrix2,shape1,shape2):
    size1 = len(matrix1)
    size2 = len(matrix2)

    size_new = shape1[2]*shape1[3]*shape2[0]*shape2[1]


    index_size1 = shape1[2]*shape2[0]
    index_size2 = shape1[3]*shape2[1]

    array_result = np.zeros(size1*size2, dtype = 'complex_')
    for lattice_number in range(shape1[0]*shape1[1]):
        for first_index in range(shape1[2]):
            for dimension in range(shape2[0]):
                for dimension2 in range(shape1[2]):
                    start_index_lattice = lattice_number * size_new

                    start_index_dim =  dimension * index_size1 + dimension2*index_size2*2

                    start_index1 = first_index * shape2[1]
                    end_index1 = (first_index+1) * shape2[1]

                    start_index = start_index_lattice + start_index_dim + start_index1
                    end_index = start_index_lattice + start_index_dim + end_index1

                    start_index_mat1_lattice = lattice_number*shape1[2]*shape1[3] +dimension2*shape1[3]
                    start_index_mat1 = start_index_mat1_lattice + first_index 
                    #end_index_mat1 = start_index_mat1_lattice+first_index+1

                    start_index_mat2 = dimension*shape2[1]
                    end_index_mat2 = (dimension+1)*shape2[1]
                    array_result[start_index: end_index] = matrix1[start_index_mat1] *matrix2[start_index_mat2:end_index_mat2]
    #                 print('start index = ' + str(start_index))
    #                 print('end index = ' + str(end_index))
    #                 print('start index mat 1 = ' + str(start_index_mat1))
    #                 #print(end_index_mat1)
    #                 print('start index mat 2= ' + str(start_index_mat2))
    #                 print('end index mat 2= ' + str(end_index_mat2))
    #                 print('\n')
    return array_result

lattice_volume2 = 20


shape1 = [lattice_volume2,1,3,3] #Corregir esta webada, el 1 ahi esta redundante!
shape2 = [2,2]

offdiagonal_spinor_x_plus = Id - Pauli[1]
matrix2 = offdiagonal_spinor_x_plus.flatten()

gauge_links = np.random.randn(lattice_volume2, nDim, Nc,Nc)
matrix1 = gauge_links[:,1,:,:].flatten()

array_reshaped = np.reshape(Kron(matrix1,matrix2,shape1,shape2), (lattice_volume2,6,6))
offdiag_x_plus = np.kron(gauge_links[:,1,:,:],offdiagonal_spinor_x_plus)
np.sqrt(np.sum((array_reshaped-offdiag_x_plus)**2))

0j

In [26]:
def Transposed_indices(nDim):
    indices = np.arange(nDim)
    repeated_indices = indices*nDim
    return (indices[:,None] + repeated_indices[None,:]).flatten()


def ConjugateTranspose(matrix,lattice_volume,nDim, tranposed_indices): 
    array_result = np.zeros(matrix.shape, dtype = 'complex_')
    size_matrix = nDim**2

    #transposed_indices = np.array([0,3,6,1,4,7,2,5,8]) for 3x3 Matrices

    for lattice_number in range(lattice_volume):
        start_index = lattice_number *size_matrix
        end_index = (lattice_number+1) *size_matrix
        indices = start_index + transposed_indices
        array_result[start_index:end_index] = np.conjugate(matrix[indices])
    return array_result

gauge_links = np.random.randn(lattice_volume2, Nc,Nc) + 1j*np.random.randn(lattice_volume2, Nc,Nc)
matrix = gauge_links.flatten()

transposed_indices = Transposed_indices(Nc)
array_result = ConjugateTranspose(matrix,lattice_volume2,Nc, transposed_indices)

array_reshaped = np.reshape(array_result,(lattice_volume2, 3,3))
transposed = np.conjugate(np.squeeze(Transpose(gauge_links)))
np.sqrt(np.sum((array_reshaped-transposed)**2))

0j

In [34]:
def applyD_flatten_flatten(lattice_volume,Nt, Nx ,gauge_links_t, gauge_links_x,m,Pauli,psi, transposed_indices):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    
    #Correct this
    offdiagonal_spinor_x_plus = (Id - Pauli[0]).flatten()
    offdiagonal_spinor_x_minus = (Id + Pauli[0]).flatten()
    offdiagonal_spinor_t_plus = (Id - Pauli[2]).flatten()
    offdiagonal_spinor_t_minus = (Id + Pauli[2]).flatten()
    
    
    

    shape_gauge_links = [Nt,Nx, Nc,Nc] #Correct this, lattice volume instead of Nt,Nx or just be consistent overall!
    gauge_links_shifted_t = Roll_t(gauge_links_t,shape_gauge_links,1,Nt,Nx)
    gauge_links_shifted_x = Roll_x(gauge_links_x,shape_gauge_links,1,Nt,Nx)
   


   

   
   
   
    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    shape_psi = [Nt,Nx, Nc*Ns]
    psi_shifted_m_t = Roll_t(psi, shape_psi, 1,Nt,Nx)
    psi_shifted_p_t = Roll_t(psi, shape_psi, -1,Nt,Nx)
    psi_shifted_m_x = Roll_x(psi, shape_psi, 1,Nt,Nx)
    psi_shifted_p_x = Roll_x(psi, shape_psi, -1,Nt,Nx)

    shape_gauge = [lattice_volume,1,Nc,Nc]
    shape_gamma = [2,2]
    offdiag_x_plus = Kron(gauge_links_x,offdiagonal_spinor_x_plus, shape_gauge, shape_gamma) #Careful with U dimension t,x
    offdiag_x_minus = Kron(ConjugateTranspose(gauge_links_shifted_x,lattice_volume, Nc, transposed_indices),offdiagonal_spinor_x_minus, shape_gauge, shape_gamma)
    offdiag_t_plus = Kron(gauge_links_t, offdiagonal_spinor_t_plus, shape_gauge, shape_gamma)
    offdiag_t_minus = Kron(ConjugateTranspose(gauge_links_shifted_t,lattice_volume, Nc, transposed_indices),offdiagonal_spinor_t_minus, shape_gauge, shape_gamma)

    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    shape_psi = [lattice_volume, Nc*Ns] #really be consistent with this shit
    shape_kron = [lattice_volume, Nc*Ns,Nc*Ns]
    time_contribution_m = MatMul(offdiag_t_minus, psi_shifted_m_t, shape_kron, shape_psi) #Do this without None
    time_contribution_p = MatMul(offdiag_t_plus, psi_shifted_p_t, shape_kron, shape_psi)
    space_contribution_m = MatMul(offdiag_x_minus, psi_shifted_m_x, shape_kron, shape_psi)
    space_contribution_p = MatMul(offdiag_x_plus, psi_shifted_p_x, shape_kron, shape_psi)



    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* (time_contribution_m +  #Squeeze is weird!
                                       time_contribution_p + space_contribution_m + space_contribution_p)
   
    return psi_new



Nt = 4
Nx = 4
lattice_volume = Nt*Nx
gauge_links = np.random.randn(lattice_volume,2, Nc,Nc)
gauge_links_t = gauge_links[:,0,:,:].flatten()
gauge_links_x = gauge_links[:,1,:,:].flatten()
m = 1
psi = np.random.randn(lattice_volume,Nc*Ns)
psi_flatten = psi.flatten()




result_flattened_operation_reshaped = np.reshape(applyD_flatten_flatten(lattice_volume,Nt, Nx ,gauge_links_t, gauge_links_x,m,Pauli,psi_flatten,transposed_indices),(lattice_volume,Nc*Ns))

result = np.reshape(applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi), (lattice_volume, Nc*Ns))

np.sqrt(np.sum((result-result_flattened_operation_reshaped)**2))

(2.766661520853722e-15+0j)